In [1]:
use Modern::Perl;
use PDL;
use Time::HiRes qw(time);

### Some functions to compare

In [2]:
sub mcpi_0 {

    my $runs  = shift;
    my $count = 0;
    
    foreach my $i (1 .. $runs){
        my $a = rand;
        my $b = rand;
        $count += 1 if (sqrt($a*$a + $b*$b) < 1);
    }

    return 4*$count/$runs;
}

sub mcpi_1 {

    my $runs  = shift;
    my $count = 0;
    
    foreach my $i (1 .. $runs){
        my $a = rand;
        my $b = rand;
        $count += 1 if (($a*$a + $b*$b) < 1);
    }

    return 4*$count/$runs;
}

sub mcpi_2 {

    my $runs  = shift;
    my $count = 0;
    
    foreach my $i (1 .. $runs){
        my $a = rand;
        my $b = rand;
        
        $count += 1 if (($a*$a + $b*$b) > 1);
    }

    return 4*(1-$count/$runs);
}

sub mcpi_3 {
    my $runs  = shift;
    my ($a,$b);
    my $count = 0;
    
    foreach my $i (1 .. $runs){
        ($a,$b) = (rand,rand);
        
        $count += 1 if (($a*$a + $b*$b) > 1);
    }

    return 4*(1-$count/$runs);
}

sub mcpi_pdl {
# two arguments
    my $runs      = shift;
    my $veclength = shift;
    my $count      = 0;
    my ($a,$b);

    foreach (1 .. $runs){
        $a = random($veclength);
        $b = random($veclength);
        $count = $count + sum(($a*$a+$b*$b) > 1);
    }


    my $total = $runs*$veclength;
    return 4*(1-$count/$total);
}


In [3]:
printf("%10.6f\n", mcpi_0(100000));
printf("%10.6f\n", mcpi_1(100000));
printf("%10.6f\n", mcpi_2(100000));
printf("%10.6f\n", mcpi_3(100000));
printf("%10.6f\n", mcpi_pdl(100,1000));

  3.135600
  3.138120
  3.137960
  3.137960
  3.145320


1


In [4]:
use Benchmark qw(cmpthese);

cmpthese(1000, {
    'mcpi_0' => sub { mcpi_0(1E4) },
    'mcpi_1' => sub { mcpi_1(1E4) },
    'mcpi_2' => sub { mcpi_2(1E4) },
    'mcpi_3' => sub { mcpi_3(1E4) },
    'mcpi_pdl' => sub { mcpi_pdl(1,1E4) },
});
0;

            (warning: too few iterations for a reliable count)
           Rate   mcpi_0   mcpi_1   mcpi_2   mcpi_3 mcpi_pdl
mcpi_0    285/s       --     -10%     -16%     -25%     -94%
mcpi_1    315/s      11%       --      -7%     -16%     -93%
mcpi_2    338/s      19%       7%       --     -10%     -93%
mcpi_3    377/s      32%      20%      12%       --     -92%
mcpi_pdl 4545/s    1495%    1341%    1245%    1105%       --


0


In [5]:
# start the clock
my $ta = time;
my $pi = mcpi_pdl(1E3,1E5);
say "time: ", time - $ta;
printf ("%10.7f\n", $pi);


time: 2.09505295753479
 3.1418064


1


## Some graphs to play with

In [6]:
my @runs = (1 .. 500);
# first run
my @pi_0 = ();
push @pi_0, mcpi_pdl(1,1E5) foreach @runs;

#second run
my @pi_1 = ();
push @pi_1, mcpi_pdl(1E1,1E5) foreach @runs;

my $PI = 3.14159265359;
IPerl->load_plugin( "ChartClicker" );
my $cc = Chart::Clicker->new(width => 523, height => 384);

my $series_0 = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [@pi_0],
);
my $series_1 = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [@pi_1],
);

my $series_pi = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [map{$PI} @runs],
);

# build the dataset
my $dataset = Chart::Clicker::Data::DataSet->new(
    series  =>      [ $series_0, $series_1, $series_pi ],
);

$cc->add_to_datasets($dataset);

$cc;


## Visualize MSE

In [7]:
my @avg_0=();
my @avg_1=();

my ($ra_0,$ra_1) = (0,0); 
foreach my $i (@runs){
    $ra_0 += $pi_0[$i-1];
    push @avg_0, $ra_0/$i;

    $ra_1 += $pi_1[$i-1];
    push @avg_1, $ra_1/$i;
}


my $cc = Chart::Clicker->new(width => 523, height => 384);

my $series_0 = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [@avg_0],
);
my $series_1 = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [@avg_1],
);

my $series_pi = Chart::Clicker::Data::Series->new(
  keys    => [@runs],
  values  => [map{$PI} @runs],
);

# build the dataset
my $dataset = Chart::Clicker::Data::DataSet->new(
    series  =>      [ $series_0, $series_1, $series_pi ],
);

$cc->add_to_datasets($dataset);

$cc;


In [9]:
printf ("%3i %10.6f %10.6f\n", $runs[$_], $avg_0[$_], $avg_1[$_]) foreach 0 .. $#runs;

  1   3.136200   3.140252
  2   3.134360   3.141098
  3   3.138307   3.140653
  4   3.139190   3.140687
  5   3.139848   3.140680
  6   3.140207   3.140551
  7   3.140143   3.140949
  8   3.140505   3.140960
  9   3.140524   3.141521
 10   3.140440   3.141650
 11   3.140156   3.141537
 12   3.139590   3.141407
 13   3.140169   3.141232
 14   3.139763   3.141410
 15   3.140275   3.141301
 16   3.140625   3.141077
 17   3.141040   3.141204
 18   3.140893   3.141175
 19   3.140522   3.141251
 20   3.140768   3.141297
 21   3.140074   3.141221
 22   3.139933   3.141354
 23   3.139950   3.141469
 24   3.140123   3.141412
 25   3.140354   3.141393
 26   3.140291   3.141372
 27   3.140523   3.141432
 28   3.140640   3.141381
 29   3.140465   3.141377
 30   3.140423   3.141359
 31   3.140465   3.141378
 32   3.140466   3.141328
 33   3.140265   3.141358
 34   3.140115   3.141348
 35   3.140133   3.141373
 36   3.140122   3.141373
 37   3.140253   3.141335
 38   3.140228   3.141349
 39   3.1400